In [2]:
import pandas as pd
import boto3

# Set the S3 bucket name and file paths
bucket_name = "sg01003movies"
s3_key1 = "movies.csv"
s3_key2 = "ratings.csv"
s3_key3 = "tmdb_5000_movies.csv"
s3_key4 = "tmdb_5000_credits.csv"
s3_key5 = "u.item"
s3_key6 = "ua.test"

# Create an S3 client
s3_client = boto3.client("s3")

# Download files and read them into DataFrames
csv_file_path1 = "movies.csv"
s3_client.download_file(bucket_name, s3_key1, csv_file_path1)

csv_file_path2 = "ratings.csv"
s3_client.download_file(bucket_name, s3_key2, csv_file_path2)

csv_file_path3 = "tmdb_5000_movies.csv"
s3_client.download_file(bucket_name, s3_key3, csv_file_path3)

csv_file_path4 = "tmdb_5000_credits.csv"
s3_client.download_file(bucket_name, s3_key4, csv_file_path4)

csv_file_path5 = "u.item"
s3_client.download_file(bucket_name, s3_key5, csv_file_path5)

csv_file_path6 = "ua.test"
s3_client.download_file(bucket_name, s3_key6, csv_file_path6)


In [13]:
movies_movielens = pd.read_csv(csv_file_path1)
ratings_movielens = pd.read_csv(csv_file_path2)
movies_tmdb = pd.read_csv(csv_file_path3)
credits = pd.read_csv(csv_file_path4)

In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
import json
import warnings
import base64
import io
from matplotlib.pyplot import imread
import codecs
from IPython.display import HTML
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')


movies_tmdb = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
movies_movielens=pd.read_csv('movies.csv')
ratings_movielens=pd.read_csv('ratings.csv')

#Pre-treatment
# Change 'genres' -> 'genre' in the mobies_movielens data frame
movies_movielens = movies_movielens.rename(columns={'genres': 'genre'})

# convert string of genres, keywords, production_companies, cast, crew column--> in tmdb_5000_movies.csv file
#genre
movies_tmdb['genres'] = movies_tmdb['genres'].apply(json.loads)
for index,i in zip(movies_tmdb.index,movies_tmdb['genres']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name'])) # 'name' 키에는 장르의 이름이 들어있습니다.
    movies_tmdb.loc[index,'genres'] = str(list1)

# keyword
movies_tmdb['keywords'] = movies_tmdb['keywords'].apply(json.loads)
for index,i in zip(movies_tmdb.index,movies_tmdb['keywords']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))
    movies_tmdb.loc[index,'keywords'] = str(list1)
    
# company
movies_tmdb['production_companies'] = movies_tmdb['production_companies'].apply(json.loads)
for index,i in zip(movies_tmdb.index,movies_tmdb['production_companies']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))
    movies_tmdb.loc[index,'production_companies'] = str(list1)

# cast
credits['cast'] = credits['cast'].apply(json.loads)
for index,i in zip(credits.index,credits['cast']):
    list1 = []
    for j in range(len(i)):
        list1.append((i[j]['name']))
    credits.loc[index,'cast'] = str(list1)

# crew  
credits['crew'] = credits['crew'].apply(json.loads)
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
credits['crew'] = credits['crew'].apply(director)
credits.rename(columns={'crew':'director'},inplace=True)

# Merge two data frames
movies_tmdb = movies_tmdb.merge(credits,left_on='id',right_on='movie_id',how='left')
movies_tmdb = movies_tmdb[['id','original_title','genres','cast','vote_average','director','keywords']]

# Remove duplicate movie title (row)
duplicate_values = movies_tmdb.duplicated(subset='original_title', keep=False)
movies_tmdb = movies_tmdb[~duplicate_values]

duplicate_values = movies_movielens.duplicated(subset='title', keep=False)
movies_movielens = movies_movielens[~duplicate_values]

# Movie title, lowercase character unification
#Remove the gap
movies_tmdb['original_title'] = movies_tmdb['original_title'].astype(str).str.lower().str.replace(' ', '')
movies_movielens['title_key'] = movies_movielens['title'].astype(str).str.lower().str.replace(' ', '')
movies_movielens['title_key']=movies_movielens.title_key.str.replace('(\(\d\d\d\d\))','',regex=True)
movies_movielens['title_key']=movies_movielens['title_key'].apply(lambda x:x.strip())
movies_movielens['title']=movies_movielens.title.str.replace('(\(\d\d\d\d\))','',regex=True)
movies_movielens['title']=movies_movielens['title'].apply(lambda x:x.strip())
common_rows = movies_tmdb[movies_tmdb['original_title'].isin(movies_movielens['title_key'].tolist())]

# Select a common row based on the movie title.
# Check the number of common rows
num_common_rows = len(common_rows)

# movies_tmdb와 movies_movielens를 병합
movies = pd.merge(movies_tmdb, movies_movielens, how='inner', left_on='original_title', right_on='title_key')
movies = movies.drop(['title'], axis=1)
movies = movies.drop(['genre'], axis=1)
duplicate_values = movies.duplicated(subset='original_title', keep=False)

# Deduplication
movies = movies[~duplicate_values]
#-->2537

rating=ratings_movielens[ratings_movielens['movieId'].isin(movies['movieId'].tolist())]
#-->48535

#Working with the Genres column
movies['genres'] = movies['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['genres'] = movies['genres'].str.split(',')

for i,j in zip(movies['genres'],movies.index):
    list2=[]
    list2=i
    list2.sort()
    movies.loc[j,'genres']=str(list2)
movies['genres'] = movies['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['genres'] = movies['genres'].str.split(',')
genreList = []
for index, row in movies.iterrows():
    genres = row["genres"]
    
    for genre in genres:
        if genre not in genreList:
            genreList.append(genre)
genreList[:10] 

#One Hot Encoding for multiple labels
def binary(genre_list):
    binaryList = []
    
    for genre in genreList:
        if genre in genre_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList
movies['genres_bin'] = movies['genres'].apply(lambda x: binary(x))


#Working with the Cast Column
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['cast'] = movies['cast'].str.split(',')

for i,j in zip(movies['cast'],movies.index):
    list2 = []
    list2 = i[:4]
    movies.loc[j,'cast'] = str(list2)
movies['cast'] = movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['cast'] = movies['cast'].str.split(',')
for i,j in zip(movies['cast'],movies.index):
    list2 = []
    list2 = i
    list2.sort()
    movies.loc[j,'cast'] = str(list2)
movies['cast']=movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')
castList = []
for index, row in movies.iterrows():
    cast = row["cast"]
    
    for i in cast:
        if i not in castList:
            castList.append(i)
def binary(cast_list):
    binaryList = []
    
    for genre in castList:
        if genre in cast_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList
movies['cast_bin'] = movies['cast'].apply(lambda x: binary(x))

#Working with Director column
def xstr(s):
    if s is None:
        return ''
    return str(s)
movies['director'] = movies['director'].apply(xstr)

directorList=[]
for i in movies['director']:
    if i not in directorList:
        directorList.append(i)
        

def binary(director_list):
    binaryList = []  
    for direct in directorList:
        if direct in director_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList


movies['director_bin'] = movies['director'].apply(lambda x: binary(x))
MLmovie=pd.read_csv('movies.csv')
move3 = pd.merge(MLmovie[['movieId', 'title']], movies, on='movieId')
movies=move3
movies.drop(columns=['original_title'],inplace=True)
movies.rename(columns={"title_x": "original_title"}, inplace=True)



movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['keywords'] = movies['keywords'].str.split(',')
for i,j in zip(movies['keywords'],movies.index):
    list2 = []
    list2 = i
    movies.loc[j,'keywords'] = str(list2)
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['keywords'] = movies['keywords'].str.split(',')
for i,j in zip(movies['keywords'],movies.index):
    list2 = []
    list2 = i
    list2.sort()
    movies.loc[j,'keywords'] = str(list2)
movies['keywords'] = movies['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['keywords'] = movies['keywords'].str.split(',')
words_list = []
for index, row in movies.iterrows():
    genres = row["keywords"]
    
    for genre in genres:
        if genre not in words_list:
            words_list.append(genre)
def binary(words):
    binaryList = []
    for genre in words_list:
        if genre in words:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList

movies['words_bin'] = movies['keywords'].apply(lambda x: binary(x))
movies = movies[(movies['vote_average']!=0)] #removing the movies with 0 score and without drector names 
movies = movies[movies['director']!='']

new_id = list(range(0,movies.shape[0]))
movies['new_id']=new_id
movies.rename(columns={"title": "original_title"}, inplace=True)
movies=movies[['original_title','genres','vote_average','genres_bin','cast_bin','new_id','director','director_bin','words_bin']]
#movies.shape

movies['genres_bin'] = movies['genres_bin'].apply(lambda x: str(list(x)))
movies['cast_bin'] = movies['cast_bin'].apply(lambda x: str(list(x)))
movies['director_bin'] = movies['director_bin'].apply(lambda x: str(list(x)))
movies['words_bin'] = movies['words_bin'].apply(lambda x: str(list(x)))
movies.to_csv('movies_features.csv', index=False)

In [16]:
# Set the S3 bucket name and file path
bucket_name = "sg01003movies"
s3_key = "movies_features.csv"

# Convert the DataFrame to CSV format and save it as a string
csv_buffer = io.StringIO()
movies.to_csv(csv_buffer, index=False)

# Create an S3 client
s3_client = boto3.client("s3")

# Upload the CSV file to S3
response = s3_client.put_object(
    Bucket=bucket_name,
    Key=s3_key,
    Body=csv_buffer.getvalue()
)

print("The Movies data has been successfully stored in S3.")



Movies 데이터가 성공적으로 S3에 저장되었습니다.


In [18]:
import pandas as pd
import numpy as np

ratings_movielens=pd.read_csv('ratings.csv')
item_frame = pd.read_csv('u.item', sep='|', header=None, encoding='latin-1', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation','Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
data = ratings_movielens

n_users = data['userId'].unique().shape[0]
n_items = data['movieId'].unique().shape[0]

n_users = data['userId'].max()
n_items = data['movieId'].max()

# n_users = max_user_id + 1
# n_items = max_movie_id + 1


test_mse_lst = []
train_mse_lst = []

K=20
steps=100
alpha=0.001
beta=0.02

# Initialize the user and item latent feature matrices
P = np.random.normal(scale=1. / K, size=(n_users + 1, K))  # n_users + 1 for the bias term
Q = np.random.normal(scale=1. / K, size=(n_items + 1, K))  # n_items + 1 for the bias term

# Initialize the biases
b_u = np.zeros(n_users + 1)
b_i = np.zeros(n_items + 1)
b = np.mean(data['rating'])

# Create a list of training samples
# contains (user_id, item_id, rating)


#test_data = test_data = pd.read_csv('data/ua.test', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

rows, cols = data.shape  # 100'000, 4
samples = [
    (data.iloc[i, 0], data.iloc[i, 1], data.iloc[i, 2])
    for i in range(rows)
]

# Perform stochastic gradient descent for number of steps
training_process = []
for step in range(steps):
    np.random.shuffle(samples)

    for i, j, r in samples:
        # Computer prediction and error
        prediction = b + b_u[i] + b_i[j] + P[i, :].dot(Q[j, :].T)

        e = (r - prediction)

        # Update biases
        b_u[i] += alpha * (e - beta * b_u[i])
        b_i[j] += alpha * (e - beta * b_i[j])

        # Update user and item latent feature matrices
        P[i, :] += alpha * (e * Q[j, :] - beta * P[i, :])
        Q[j, :] += alpha * (e * P[i, :] - beta * Q[j, :])

    # Compute total mean squared error
    mse = 0

    for i, j, r in samples:
        prediction = b + b_u[i] + b_i[j] + P[i, :].dot(Q[j, :].T)
        mse += (r - prediction) ** 2

    mse /= len(samples)
    training_process.append((step, mse))
    
    # if (step % 10) == 0:
    #     print("Iteration: %d ; error = %.4f" % (step, mse))

    #     mf_mse = self.test_matrix_factorisation(P, Q, b_u, b_i, b)
    #     test_mse_lst.append(mf_mse)
    #     train_mse_lst.append(mse)
    #     print("TEST MSE - MF MSE: {}".format(mf_mse))
   
        
### test MF
test_data = pd.read_csv('ua.test', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
rows, cols = test_data.shape
samples = [
    (test_data.iloc[i, 0], test_data.iloc[i, 1], test_data.iloc[i, 2])
    for i in range(rows)
]
# Compute MSE for the test samples
mse = 0

for i, j, r in samples:
    try: 
        prediction = b + b_u[i] + b_i[j] + P[i, :].dot(Q[j, :].T)
        mse += (r - prediction) ** 2
    except IndexError as e:
        print(f"IndexError: {e}. Skipping...")
mse /= len(samples)




IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 611 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 612 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 612 is out of bounds for axis 0 with size 611. Skipping...
IndexError: index 612 is out of bounds for axis 0 with size 611. Skipping...

In [22]:
import pickle
import boto3

# Set the S3 bucket name and file paths
bucket_name = "sg01003movies"
s3_folder = "user/"
s3_key1 = s3_folder + "samples.pkl"
s3_key2 = s3_folder + "b.pkl"
s3_key3 = s3_folder + "b_u.pkl"
s3_key4 = s3_folder + "b_i.pkl"
s3_key5 = s3_folder + "P.pkl"
s3_key6 = s3_folder + "Q.pkl"

# Local file paths to store the data
pkl_file_path1 = "data/samples.pkl"
pkl_file_path2 = "data/b.pkl"
pkl_file_path3 = "data/b_u.pkl"
pkl_file_path4 = "data/b_i.pkl"
pkl_file_path5 = "data/P.pkl"
pkl_file_path6 = "data/Q.pkl"

# Data storage
with open(pkl_file_path1, "wb") as f:
    pickle.dump(samples, f)

with open(pkl_file_path2, "wb") as f:
    pickle.dump(b, f)

with open(pkl_file_path3, "wb") as f:
    pickle.dump(b_u, f)

with open(pkl_file_path4, "wb") as f:
    pickle.dump(b_i, f)

with open(pkl_file_path5, "wb") as f:
    pickle.dump(P, f)

with open(pkl_file_path6, "wb") as f:
    pickle.dump(Q, f)

# Create an S3 client
s3_client = boto3.client("s3")

# Upload files
s3_client.upload_file(pkl_file_path1, bucket_name, s3_key1)
s3_client.upload_file(pkl_file_path2, bucket_name, s3_key2)
s3_client.upload_file(pkl_file_path3, bucket_name, s3_key3)
s3_client.upload_file(pkl_file_path4, bucket_name, s3_key4)
s3_client.upload_file(pkl_file_path5, bucket_name, s3_key5)
s3_client.upload_file(pkl_file_path6, bucket_name, s3_key6)

print("The data has been uploaded to S3.")


데이터를 S3에 업로드했습니다.
